In [18]:
import numpy as np
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D

from keras.applications import VGG16, VGG19

In [19]:
input_shape = [224, 224, 3]
# defining vgg model
def build_model():
    model = Sequential([
        Conv2D(64, (3, 3), input_shape=input_shape, padding='same', activation='relu'),
        Conv2D(64, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        Conv2D(128, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        Conv2D(256, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(2, 2), strides=(2, 2)),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        Conv2D(512, (3, 3), activation='relu', padding='same'),
        MaxPooling2D(pool_size=(3, 3), strides=(2, 2)),
        Flatten(),
        Dense(4096, activation='relu'),
        Dense(4096, activation='relu'),
        Dense(1000, activation='softmax')
    ])
    return model        

In [20]:
model = build_model()
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_91 (Conv2D)           (None, 224, 224, 64)      1792      
_________________________________________________________________
conv2d_92 (Conv2D)           (None, 224, 224, 64)      36928     
_________________________________________________________________
max_pooling2d_35 (MaxPooling (None, 112, 112, 64)      0         
_________________________________________________________________
conv2d_93 (Conv2D)           (None, 112, 112, 128)     73856     
_________________________________________________________________
conv2d_94 (Conv2D)           (None, 112, 112, 128)     147584    
_________________________________________________________________
max_pooling2d_36 (MaxPooling (None, 56, 56, 128)       0         
_________________________________________________________________
conv2d_95 (Conv2D)           (None, 56, 56, 256)       295168    
__________